In [1]:
import pandas as pd
import random
import re
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.nn import Sigmoid
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler
from torch.nn import (Linear, Sigmoid, Softplus, ELU, ReLU,GELU ,BatchNorm1d, ModuleList, Sequential, Tanh)
from Graph_Dataset import GraphDataset, Atomgraph_collate
from model_MBVGNN import Model, MBVGNN

In [2]:
path = './val/'
database = 'Eg'
filename = 'my_graph_data_MP_test1_8_12_100_000'
target_name = "average_voltage"
cutoff = int(8)
n_Gaussian = int(128)
n_grid_K = int(4)
new_data = GraphDataset(path, filename, database, target_name, cutoff,n_grid_K, n_Gaussian)

The number of files = 1
loading : ./val/npz/my_graph_data_MP_test1_8_12_100_000.npz
load successed, final volume : 307
the number of valid targets = 307
start to constructe Graph
finish constructe the graph
The number of valid graphs = 307


In [3]:
n_node_feat = int(new_data.graph_data[0].nodes.shape[1])
n_hidden_feat = int(216)
n_GCN_feat = int(216)
conv_bias = False
N_block = int(3)
n_MLP_LR = int(3)
cutoff = int(8)
max_nei = int(12)
n_Gaussian = int(128)
n_grid_K = int(4)
node_activation = 'Sigmoid'
MLP_activation = 'ELU'

use_node_batch_norm = True
use_edge_batch_norm = True

# 创建CDGNN模型实例
model = MBVGNN(n_node_feat, n_hidden_feat, n_GCN_feat, conv_bias, N_block, node_activation, MLP_activation,
              use_node_batch_norm, use_edge_batch_norm, cutoff, max_nei, n_MLP_LR, n_grid_K,n_Gaussian)

# 加载预训练的模型权重
model.load_state_dict(torch.load('model/model_3_8_12.pth'))

<All keys matched successfully>

In [4]:
dataloader_param = dict()
dataloader_param["collate_fn"] = Atomgraph_collate
dataloader_param["batch_size"] = 30

dataset = DataLoader(new_data,pin_memory=True,**dataloader_param)

In [5]:
MSELoss = torch.nn.MSELoss
L1Loss =  torch.nn.L1Loss
optimizer = optim.Adam(model.parameters())
aa = Model(device='cuda',
      model=model,
      name = "3_8_12",
      optimizer = optimizer,
      scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[1]),
      clip_value=None,
      metrics=[('loss', MSELoss()), ('mae', L1Loss())])

In [6]:
all_outputs, all_targets, all_graph_vec = aa.evaluate(dataset)

test loss: 0.0748 mae: 0.1636


In [7]:
df_predictions = pd.DataFrame({"prediction": all_outputs, "target": all_targets})
target_names = pd.read_csv("./val/targets_Eg.csv")

In [8]:
prediction_dataframe = pd.concat([target_names,df_predictions],axis=1)
prediction_dataframe.drop(columns=['average_voltage'], inplace=True)

In [9]:
prediction_dataframe

,id,prediction,target
0,mp-1006112,-0.010416,-0.876627
1,mp-1186037,0.043702,-0.661558
2,mp-1186073,-0.094699,-0.555672
3,mp-1120777,1.618308,-0.060390
4,mp-1190392,0.378797,0.440480
...,...,...,...
302,mp-1094125,5.184664,5.233230
303,mp-560780,5.247369,5.320455
304,mp-553997,5.278298,5.518749
305,mp-752506,5.880891,6.166820


In [10]:
prediction_dataframe.to_csv("data/prediction_average_voltage.csv",index=False)